<div style="color:pink">
hier möchte ich einmal versuchen, die Pixel, die von Bots gesetzt wurden, nachzuzeichnen
</div>

In [ ]:
#alle Schritte, die nötig waren, um die Bots einzustufen
from src.data.dataset_functions import get_dataframe
import pyspark.sql.functions as F

dataFrame = get_dataframe(True)

timeFrame = dataFrame.alias('df1').join(dataFrame.alias('df2'),F.col('df1.user_id') == F.col('df2.user_id'),'inner')
timeFrame = timeFrame.where('df2.t > df1.t')
timeFrame = timeFrame.orderBy(['df1.user_id', 'df1.t'])
timeFrame = timeFrame.select(F.col('df1.user_id'),F.col('df1.t').alias('t1'),F.col('df2.t').alias('t2'),(F.col('df2.t') - F.col('df1.t')).alias('diff'))
timeReduced = timeFrame.dropDuplicates(['user_id', 't1'])

bot = timeReduced.alias('tdr1').join(timeReduced.alias('tdr2'),F.col('tdr1.user_id') == F.col('tdr2.user_id'),'inner')
bot = bot.where('tdr1.diff == tdr2.diff').where('tdr1.t2 == tdr2.t1')
bot = bot.select('tdr1.user_id','tdr1.diff')
bot = bot.dropDuplicates(['user_id'])

In [ ]:
botsTotal = dataFrame.alias('df').join(bot.alias('bd'),F.col('df.user_id') == F.col('bd.user_id'), 'inner')
botsTotal = botsTotal.select(F.col('df.user_id'),F.col('df.x'),F.col('df.y'),F.col('df.t'),F.col('pixel_color'))
botsTotal = botsTotal.orderBy(['user_id', 't'])
botsTotal = botsTotal.dropDuplicates(['user_id', 't']) #jeder Nutzer kann zu jeder Zeit nur einen Pixel gesetzt haben  
botsTotal = botsTotal.drop('t')

<div style="color:pink">
<s>-aktuell verzweifle ich noch daran, die Werte aus dem dataframe iterativ fürs Zeichnen auf mein Canvas zu benutzen <br>
-aus mir unbekannten Gründen nimmt python nämlich nicht die tatsächlichen Werte aus den Spalten, sondern lediglich die Namen der Spalten selbst<br>
-weiterhin muss ich noch irgendwie hinbekommen, dass die hex-codes der farben in die richtigen rgb-werte umgewandelt werden <br> </s>
<br>
Geschafft! :))))
</div>

In [ ]:
from PIL import Image
from src.visualization.visuialization_functions import hex_to_rgb
  
#ich konvertiere den Datensatz, damit mir das Iterieren etwas leichter fällt
pandasDF = botsTotal.toPandas()
canvas = Image.new("RGB", (2000,2000), "WHITE")

#Schritt für Schritt wird wirklich jeder einzelne Pixel geändert
for index, row in pandasDF.iterrows():
    canvas.putpixel((row['x'],row['y']), hex_to_rgb(row['pixel_color']))

canvas.show()
canvas.save('../reports/figures/bots-canvas.jpg')  